In [1]:
import sys
#sys.path.insert(0, "/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/CS696DS-Oracle-Retrieving-Code-Explanations/coir-main")
#sys.path.remove("/work/pi_wenlongzhao_umass_edu/27/CS696DS-Oracle-Retrieving-Code-Explanations/coir-main")

#print(sys.path)


In [2]:
if 'coir' in sys.modules:
    del sys.modules['coir']

In [3]:
from coir.data_loader import get_tasks
from coir.evaluation import COIR
from coir.models import YourCustomDEModel
import pandas as pd

/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/envs/virtual-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def load_data(dataset_name):
    tasks = get_tasks(tasks=[dataset_name])
    return tasks

In [5]:
def add_expl(tasks, dataset_name, explanation_df_path, col_name='explanation_deepseek_1_cleaned'):
    corpus, queries, qrels = tasks[dataset_name]
    expl_df = pd.read_csv(explanation_df_path)
    expl_df.rename(columns={"query_id": "query-id", "corpus_id": "corpus-id"}, inplace=True)
    for _, row in expl_df.iterrows():
        corpus_id = row['corpus-id']
        explanation = row[col_name]

        if corpus_id in corpus and explanation and explanation.strip():
            corpus[corpus_id]['text'] = explanation  

    corpus = {doc_id: doc for doc_id, doc in corpus.items() if 'text' in doc and doc['text'].strip()}
    print(corpus['c1'])
    tasks[dataset_name] = (corpus, queries, qrels)

    print(f"Total docs in corpus after replacement: {len(tasks[dataset_name][0])}")
    missing_text = sum(1 for doc in tasks[dataset_name][0].values() if not doc.get('text'))
    print(f"Number of docs still missing 'text': {missing_text}")

In [14]:
def add_expl_2queries(tasks, dataset_name, explanation_df_path, col_name='explanation_deepseek_1_cleaned'):
    corpus, queries, qrels = tasks[dataset_name]
    expl_df = pd.read_csv(explanation_df_path)
    expl_df.rename(columns={"query_id": "query-id", "corpus_id": "corpus-id"}, inplace=True)
    for _, row in expl_df.iterrows():
        query_id = row['query-id']
        explanation = row[col_name]

        if query_id in queries and explanation and explanation.strip():
            queries[query_id] = explanation  

    queries = {doc_id: doc for doc_id, doc in queries.items()}
    print(queries['q265734'])
    tasks[dataset_name] = (corpus, queries, qrels)

    print(f"Total docs in q after replacement: {len(tasks[dataset_name][1])}")

In [6]:
def run(model_name, tasks, llm_name, retrieval_name, dataset_name):
    model = YourCustomDEModel(model_name=model_name)

    evaluation = COIR(tasks=tasks,batch_size=256)
    if(retrieval_name == "bm25"):
        results = evaluation.run(model, output_folder=f"results/{dataset_name}/{llm_name}/{retrieval_name}")
    else:
        results = evaluation.run(model, output_folder=f"results/{dataset_name}/{llm_name}/{retrieval_name}/{model_name}")
    print(results)

In [7]:
cosqa_expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/Cosqa/CoSQA_explanations_query_code.csv'
csn_expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/explanations/CodeSearchNet_Python_deepseek_allSplits.csv'

In [8]:
dataset_name = "CodeSearchNet-python"

In [9]:
tasks = load_data(dataset_name)

in tasks 
fetched data from hf
data loader init


280310it [00:01, 201213.94it/s]


In [16]:
corpus, queries, qrels = tasks[dataset_name]

In [17]:
corpus['c265731']

{'text': 'Gets the latest state of a long-running operation in Google Storage\n        Transfer Service.\n\n        :param job_name: (Required) Name of the job to be fetched\n        :type job_name: str\n        :param project_id: (Optional) the ID of the project that owns the Transfer\n            Job. If set to None or missing, the default project_id from the GCP\n            connection is used.\n        :type project_id: str\n        :return: Transfer Job\n        :rtype: dict',
 'title': ''}

In [18]:
queries['q265734']

'The docstring and code snippets represent a function that parses an XML string and extracts URLs from it. The function processes each element of the XML to collect URL data.'

In [15]:
add_expl_2queries(tasks, "CodeSearchNet-python", csn_expl_path)

The docstring and code snippets represent a function that parses an XML string and extracts URLs from it. The function processes each element of the XML to collect URL data.
Total docs in q after replacement: 14918


In [19]:
run("BAAI/bge-base-en", tasks, "deepseek1", "dres", "codesearchnet")

YourCustomDEModel init
COIR init!
in evaluation.py: loading up dres

in evaluation.py: retrieving

in beir/retrieval/evaluation.py: loading up search

len of queries:  14918


Encoding batches: 100%|██████████| 59/59 [02:07<00:00,  2.16s/batch]


len of corpus:  280310


Encoding batches: 100%|██████████| 119/119 [00:07<00:00, 16.16batch/s]


in evaluation.py: evaluating

in beir/retrieval/evaluation.py: evaluating now

Retrieval evaluation results saved to results/codesearchnet/deepseek1/dres/BAAI/bge-base-en
{'CodeSearchNet-python': {'NDCG': {'NDCG@1': 0.57897, 'NDCG@3': 0.64679, 'NDCG@5': 0.66208, 'NDCG@10': 0.67648, 'NDCG@100': 0.69883, 'NDCG@1000': 0.70687}, 'MAP': {'MAP@1': 0.57897, 'MAP@3': 0.63054, 'MAP@5': 0.63906, 'MAP@10': 0.64502, 'MAP@100': 0.64943, 'MAP@1000': 0.64971}, 'Recall': {'Recall@1': 0.57897, 'Recall@3': 0.69366, 'Recall@5': 0.73059, 'Recall@10': 0.77504, 'Recall@100': 0.88169, 'Recall@1000': 0.94644}, 'Precision': {'P@1': 0.57897, 'P@3': 0.23122, 'P@5': 0.14612, 'P@10': 0.0775, 'P@100': 0.00882, 'P@1000': 0.00095}}}


In [22]:
# tasks = load_data("CodeSearchNet-python")
# run("BAAI/bge-base-en", tasks, "baseline", "dres", "codesearchnet")